In [ ]:
# this notebook loads and manipulates E-Corp's 2-week dataset that was given to us, uses
# collaborative filtering to train an item to item rec engine, generates and saves the
# recommendations into Excel

__author__ = 'Josh Bacher'
__email__ = 'bacherjp@gmail.com'
__website__ = 'https://github.com/jpbacher'


import numpy as np
import pandas as pd


class Data:
    
    def __init__(self, file):
        '''initiate the Data class'''
        
        self.file = self._read_file(file)
        
    def _read_file(self, file):
        
        
        
    